In [2]:
import requests
from bs4 import BeautifulSoup
import datetime
import ipywidgets as widgets
from ipywidgets import Label, Layout

In [3]:
Dollar_rub = 'https://www.google.ru/search?newwindow=1&sxsrf=ALeKk00CGIqsvYlKeAslskUdBGGt11rFIA%3A1605979081454&source=hp&ei=yUu5X6aSGai5gwf9pLOYCw&q=dollar+rub&oq=dollar+rub&gs_lcp=CgZwc3ktYWIQAzIFCAAQywEyBQgAEMsBMgUIABDLATIFCAAQywEyBQgAEMsBMgIIADIFCAAQywEyBQgAEMsBMgUIABDLATIFCAAQywE6BAgjECc6BQgAELEDOggIABCxAxCDAToFCC4QsQM6BwgAEAoQywFQkAZY5idgwCloAHAAeACAAUuIAZsFkgECMTCYAQCgAQGqAQdnd3Mtd2l6&sclient=psy-ab&ved=0ahUKEwjmmqilkpTtAhWo3OAKHX3SDLMQ4dUDCAc&uact=5'
Euro_rub = 'https://www.google.ru/search?newwindow=1&sxsrf=ALeKk00mr36I6KAV8ahHu38LzCKOKqFNtg%3A1605979087533&ei=z0u5X4CPIMSQrgS8qZaQBQ&q=euro+rub&oq=euro+rub&gs_lcp=CgZwc3ktYWIQAzIHCAAQsQMQQzIFCAAQywEyBQgAEMsBMgUIABDLATIFCAAQywEyBAgAEEMyBQgAEMsBMgUIABDLATIECAAQCjIHCAAQChDLAToECAAQRzoGCAAQBxAeOggIABAHEAoQHjoCCABQ9LABWPS4AWD9vAFoAHADeACAAVmIAa0CkgEBNJgBAKABAaoBB2d3cy13aXrIAQjAAQE&sclient=psy-ab&ved=0ahUKEwjAsp2okpTtAhVEiIsKHbyUBVIQ4dUDCA0&uact=5'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}

## курс доллара
full_page = requests.get(Dollar_rub, headers=headers )
soup = BeautifulSoup(full_page.content, 'html.parser')
convert = soup.findAll("span", {"class":"DFlfde", "class": "SwHCTb"})
rub_dollar = convert[0].text.replace(",",".")
## курс евро
full_page = requests.get(Euro_rub, headers=headers )
soup = BeautifulSoup(full_page.content, 'html.parser')
convert = soup.findAll("span", {"class":"DFlfde", "class": "SwHCTb"})
rub_euro = convert[0].text.replace(",",".")

#время и дата
day = datetime.datetime.now()
print('Dollar exchange rate for ' + day.strftime("%A, %d. %B %Y %I:%M %p") + " on the Central Bank : " + rub_dollar )
print('Euro exchange rate for ' +day.strftime("%A, %d. %B %Y %I:%M %p") + " on the Central Bank : " + rub_euro )

In [4]:
## где поменять

d = {"sell_dollar":0, "buy_dollar":0, "sell_euro":0, "buy_euro":0}

def parse_block(item):
    value = item.select('div.table-flex__rate.font-size-large.color-pumpkin-orange')
    if value != []:
        name = item.find_all("a", {"class":"font-bold" })[0].text
        for i in value:
            usd_eur = i.get('data-currencies-code')
            val = i.get('data-currencies-rate-buy')
            if val != None:
                if usd_eur == 'USD':
                    d['sell_dollar'] = [val, name]
                else :
                    d["sell_euro"] = [val, name]
        for i in value:
            usd_eur = i.get('data-currencies-code')
            val = i.get('data-currencies-rate-sell')
            if val != None:
                if usd_eur == 'USD':
                    d['buy_dollar'] = [val, name]
                else :
                    d["buy_euro"] = [val, name]


Exchange = 'https://www.banki.ru/products/currency/cash/sankt-peterburg/'

full_page = requests.get(Exchange, headers=headers )
soup = BeautifulSoup(full_page.content, 'html.parser')
container = soup.select('div.table-flex__row.item.calculator-hover-icon__container')
for item in container:
    block = parse_block(item)

print('Better to buy dollar in "' + d['buy_dollar'][1] + '"; exchange rate: ' + d['buy_dollar'][0] )
print('Better to sell dollar in "' + d['sell_dollar'][1] + '"; exchange rate: ' + d['sell_dollar'][0] )
print()
print('Better to buy euro in "' + d['buy_euro'][1] + '"; exchange rate: ' + d['buy_euro'][0] )
print('Better to sell euro in "' + d['sell_euro'][1] + '"; exchange rate: ' + d['sell_euro'][0] )

In [5]:
style = {'description_width': 'initial'}

w1 = widgets.Dropdown(
    options=['USD/RUB', 'EUR/RUB', 'RUB/USD', 'RUB/EUR' ],
    value='USD/RUB',
    description='Выберите валюту которую хотите купить/продать:',
    disabled=False,
    layout=Layout(width='40%', height='30px'), style=style)
w2 = widgets.FloatText(description='Выберите сумму, которую хотите обменять',layout=Layout(width='40%', height='30px'), style=style)


def pp(val1, val2):
    if str(val1) == 'USD/RUB':
        print(float('{:.3f}'.format(val2 / float(d['buy_dollar'][0]))),"$" + ' in "' + d['buy_dollar'][1] +'"')
    elif str(val1) == 'EUR/RUB':
        print(float('{:.3f}'.format(val2 / float(d['buy_euro'][0]))), "€" + ' in "' + d['buy_euro'][1] +'"')
    elif str(val1) == 'RUB/USD':
        print(float('{:.3f}'.format(val2 * float(d['sell_dollar'][0]))),"₽"  + ' in "' + d['sell_dollar'][1] +'"')
    elif str(val1) == 'RUB/EUR':
        print(float('{:.3f}'.format(val2 * float(d['sell_euro'][0]))),"₽" + ' in "' + d['sell_euro'][1] +'"')


widgets.interact(pp, val1=w1, val2=w2) 